LSTM Model with incremental learning to improve 15mins timeframe predictions of crypto assets

In [ ]:
!pip install ccxt pandas numpy scikit-learn tensorflow pandas-ta

In [ ]:
import os
import numpy as np
import pandas as pd
import ccxt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import joblib

# File paths for model and scalers
MODEL_FILE = "lstm_model.h5"
SCALER_FILE = "scaler.pkl"
TARGET_SCALER_FILE = "target_scaler.pkl"

# Fetch historical data
def fetch_historical_data(exchange, symbol, limit=200):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe='15m', limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Preprocess data
def preprocess_data(df, sequence_length=50):
    # Feature scaling
    scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()

    features = df[['open', 'high', 'low', 'close', 'volume']]
    scaled_features = scaler.fit_transform(features)

    # Normalize target column (close price)
    scaled_target = target_scaler.fit_transform(df[['close']])

    # Save scalers for future use
    joblib.dump(scaler, SCALER_FILE)
    joblib.dump(target_scaler, TARGET_SCALER_FILE)

    # Create sequences
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_target[i + sequence_length])
    return np.array(X), np.array(y), scaler, target_scaler

# Load or create LSTM model
# Load or create LSTM model
def create_or_load_model(input_shape):
    if os.path.exists(MODEL_FILE):
        try:
            print("Loading existing LSTM model...")
            model = load_model(MODEL_FILE, custom_objects={"mse": "mse"})
            # Check for shape mismatch
            if model.input_shape[1:] != input_shape:
                print("Model input shape mismatch. Recreating the model...")
                model = None
        except Exception as e:
            print(f"Error loading model: {e}. Recreating the model...")
            model = None
    else:
        model = None

    if model is None:
        print("Creating a new LSTM model...")
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=input_shape),
            LSTM(50),
            Dense(1)
        ])

    # Reset optimizer and recompile
    print("Recompiling the model with a new optimizer...")
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model


# Train the model
def train_model(model, X, y, epochs=10, batch_size=32):
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)
    model.save(MODEL_FILE)
    return model

# Predict with LSTM
def predict(model, X, target_scaler):
    predictions = model.predict(X)
    return target_scaler.inverse_transform(predictions)

# Generate trading signals
def generate_signals(exchange, assets):
    sequence_length = 50
    results = []

    for asset in assets:
        print(f"Processing {asset}...")
        df = fetch_historical_data(exchange, asset)
        if df is None or len(df) < sequence_length:
            print(f"Insufficient data for {asset}. Skipping.")
            continue

        X, y, scaler, target_scaler = preprocess_data(df, sequence_length)
        model = create_or_load_model((sequence_length, X.shape[2]))
        model = train_model(model, X, y)

        # Use the latest sequence for prediction
        latest_sequence = X[-1].reshape(1, *X.shape[1:])
        predicted_price = predict(model, latest_sequence, target_scaler)

        current_price = df['close'].iloc[-1]
        signal = "Buy" if predicted_price > current_price else "Sell"

        results.append({
            "asset": asset,
            "current_price": current_price,
            "predicted_price": predicted_price[0][0],
            "signal": signal
        })

    return results

# Main script
exchange = ccxt.kucoin()
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "XRP/USDT", "LINK/USDT", "COOKIE/USDT",
          "SOL/USDT" , "AVAX/USDT" , "CRV/USDT" , "CFX/USDT" , "INJ/USDT" , "DOGE/USDT" ,
          "KAS/USDT" , "NEAR/USDT" , "FET/USDT"]

signals = generate_signals(exchange, assets)

# Display results
print("\n--- Trading Signals ---")
for signal in signals:
    print(f"Asset: {signal['asset']}")
    print(f"  Current Price: ${signal['current_price']:.2f}")
    print(f"  Predicted Price: ${signal['predicted_price']:.2f}")
    print(f"  Signal: {signal['signal']}")


Processing BTC/USDT...


Loading existing LSTM model...
Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - loss: 0.0028
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0022
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0020
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0020
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0018
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0016
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0021
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - loss: 0.0020
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0016
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0019


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
Processing ETH/USDT...
Loading existing LSTM model...


Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.0017
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0015
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0013
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0014
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0014
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0012
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0016
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0013
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0015
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0011


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
Processing ADA/USDT...


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
Processing NEAR/USDT...
Loading existing LSTM model...


Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.0019
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0016
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0015
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0014
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0014
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0014
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0014
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0017
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0014
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0016


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
Processing FET/USDT...
Loading existing LSTM model...


Recompiling the model with a new optimizer...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 0.0018
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0019
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0012
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0015
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0013
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0013
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0012
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0011
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0011
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0013


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step

--- Trading Signals ---
Asset: BTC/USDT
  Current Price: $102043.00
  Predicted Price: $101892.21
  Signal: Sell
Asset: ETH/USDT
  Current Price: $3161.35
  Predicted Price: $3160.48
  Signal: Sell
Asset: ADA/USDT
  Current Price: $0.95
  Predicted Price: $0.95
  Signal: Buy
Asset: XRP/USDT
  Current Price: $3.13
  Predicted Price: $3.11
  Signal: Sell
Asset: LINK/USDT
  Current Price: $23.54
  Predicted Price: $23.57
  Signal: Buy
Asset: COOKIE/USDT
  Current Price: $0.27
  Predicted Price: $0.27
  Signal: Buy
Asset: SOL/USDT
  Current Price: $234.13
  Predicted Price: $234.35
  Signal: Buy
Asset: AVAX/USDT
  Current Price: $33.52
  Predicted Price: $33.76
  Signal: Buy
Asset: CRV/USDT
  Current Price: $0.70
  Predicted Price: $0.71
  Signal: Buy
Asset: CFX/USDT
  Current Price: $0.15
  Predicted Price: $0.15
  Signal: Buy
Asset: INJ/USDT
  Current Price: $18.58
  Predicted Price: $18.65
  Signal: Buy
Asset: DOGE/USDT
  Current Price: $0.33
  Pr

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Initialize KuCoin exchange using CCXT
exchange = ccxt.kucoin()

# Fetch historical data from KuCoin
def fetch_historical_data(symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

# Add technical indicators
def add_technical_indicators(df):
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Preprocess data
def preprocess_data(df, sequence_length=50):
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict price
def predict_price(symbol, sequence_length=50):
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    rsi = df_15m['RSI'].iloc[-1]
    atr = df_15m['ATR'].iloc[-1]
    obv = df_15m['OBV'].iloc[-1]

    return {
        'symbol': symbol,
        'current_price': df_15m['close'].iloc[-1],
        '15m_prediction': predicted_price_15m,
        '4h_prediction': predicted_price_4h,
        'RSI': rsi,
        'ATR': atr,
        'OBV': obv
    }

# Main function
symbols = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT']  # Replace with desired trading pairs
for symbol in symbols:
    result = predict_price(symbol)
    print(f"Symbol: {result['symbol']}")
    print(f"Current Price: {result['current_price']:.2f}")
    print(f"15m Prediction: {result['15m_prediction']:.2f}")
    print(f"4h Prediction: {result['4h_prediction']:.2f}")
    print(f"RSI: {result['RSI']:.2f}")
    print(f"ATR: {result['ATR']:.2f}")
    print(f"OBV: {result['OBV']:.2f}")
    print("\n")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.1787
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - loss: 0.0319
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0135
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0080
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0061
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.1887
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0234
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0173
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0122
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0085
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
Symbol: BTC/USDT
Current Price: 102084.80
15m Prediction: 102956.66
4h Prediction: 100642.98
RSI: 40.64
ATR: 369.76
OBV: -664.89


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - loss: 0.1273
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0185
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0077
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0055
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0038
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - loss: 0.0684
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0171
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0091
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.0072
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0056
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
Symbol: ETH/USDT
Current Price: 3161.98
15m Prediction: 3190.18
4h Prediction: 3172.17
RSI: 37.29
ATR: 12.64
OBV: -5192.35


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.1234
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0254
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0065
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.0042
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.0035
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.0394
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0094
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0070
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0057
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0046
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
Symbol: ADA/USDT
Current Price: 0.95
15m Prediction: 0.96
4h Prediction: 0.92
RSI: 44.33
ATR: 0.01
OBV: -8579188.64


